In [32]:
import warnings
warnings.filterwarnings('ignore')
import os, sys
src_path = os.path.join(os.path.abspath('..'), 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

#from src.local_models.llm import MyLLM, LlamaCPPLLM
from llama_index.llms.openai import OpenAI
from local_models.embeddings import get_embed_model

from data_loader.splitting import split_by_md_headers
from data_loader.parsing import get_html, extract_md_tables

from llama_index.core import Document
from data_loader.chunking import chunk_docs_standalone
from llama_index.core import VectorStoreIndex
from llama_index.core import PromptTemplate

from data_loader.load_from_dir import rebuild_index
from utils import load_prompt



from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.openai import OpenAIAgent


from dotenv import load_dotenv
import os

In [26]:
load_dotenv(override=True)

True

In [62]:
embed_model = get_embed_model(model_name=os.environ['embed_path'],  model_kwargs={'device': 'cpu'}, encode_kwargs = {'normalize_embeddings': True})
llm = OpenAI()

In [111]:
#first-time build

df = split_by_md_headers('../data/第三部分.md')
df['text_html'] = df['content'].apply(lambda x: get_html(x))
df = extract_md_tables(df)

In [112]:
df

,Header 1,Header 2,Header 3,Header 4,Header 5,content,is_table,text,table
0,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,17 试验作业,17.1 一般要求,NaN,- 电气试验的具体标准、方法等应遵照 GB 26861—2011、DL/T 596—2005...,0,* 电气试验的具体标准、方法等应遵照 GB 26861—2011、DL/T 596—2005...,
1,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,17 试验作业,17.2 高压试验,NaN,- 同一电气连接部分，检修和高压试验工作不能同时进行。许可高压试验前，应将其他检修工作间断并...,0,* 同一电气连接部分，检修和高压试验工作不能同时进行。许可高压试验前，应将其他检修工作间断并...,
2,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,18 电气测量作业,18.1 一般要求,NaN,- 雷电天气时，禁止测量接地电阻、设备绝缘电阻及进行高压侧核相工作。沿线出现雷雨天气时不应进...,0,* 雷电天气时，禁止测量接地电阻、设备绝缘电阻及进行高压侧核相工作。沿线出现雷雨天气时不应进...,
3,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,18 电气测量作业,18.2 使用携带型仪器测量,NaN,- 除使用特殊仪器外，所有使用携带型仪器的测量工作，均应在电流互感器和电压互感器的二次侧进行...,0,* 除使用特殊仪器外，所有使用携带型仪器的测量工作，均应在电流互感器和电压互感器的二次侧进行...,
4,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,18 电气测量作业,18.3 使用钳形电流表测量,NaN,- 在高压回路上测量时，不应用导线从钳形电流表另接表计测量。\n- 测量低压熔断器和水平排列...,0,* 在高压回路上测量时，不应用导线从钳形电流表另接表计测量。\n* 测量低压熔断器和水平排列...,
5,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,18 电气测量作业,18.4 使用绝缘电阻表测量,NaN,- 测量用的导线，应使用相应的绝缘导线，其端部应有绝缘套。\n- 测量设备绝缘电阻时，应将被...,0,* 测量用的导线，应使用相应的绝缘导线，其端部应有绝缘套。\n* 测量设备绝缘电阻时，应将被...,
6,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,18 电气测量作业,18.5 高压线路使用接地电阻表测量,NaN,- 测量杆塔、配电变压器和避雷器的接地电阻，可在线路和设备带电的情况下进行。解开或恢复配电变...,0,* 测量杆塔、配电变压器和避雷器的接地电阻，可在线路和设备带电的情况下进行。解开或恢复配电变...,
7,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,18 电气测量作业,18.6 核相作业,NaN,- 高压侧核相时应戴绝缘手套，使用相应电压等级的核相器，并逐相进行。\n- 二次侧核相时，应...,0,* 高压侧核相时应戴绝缘手套，使用相应电压等级的核相器，并逐相进行。\n* 二次侧核相时，应...,
8,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,18 电气测量作业,18.7 线路工频参数测量,NaN,- 接测试引线前，应切断测试主回路电源，对被测线路验电、接地，接测试引线完成后再拆除接地。\...,0,* 接测试引线前，应切断测试主回路电源，对被测线路验电、接地，接测试引线完成后再拆除接地。\...,
9,中国南方电网有限责任公司电力安全工作规程,第 3 部分 专项作业,19 水轮机作业,NaN,NaN,1. 进入水轮机内工作时，应采取以下措施：\na) 关闭进水闸门，排除输水钢管内积水，并保持...,0,1. 进入水轮机内工作时，应采取以下措施：\na) 关闭进水闸门，排除输水钢管内积水，并保持...,


In [113]:
table_docs = [Document(text=tbl, metadata={}) for tbl in df[df['is_table']==1]['table'].values]

text_docs = [Document(text=tbl, metadata={}) for tbl in df[df['is_table']==0]['text'].values]
text_nodes = [chunk_docs_standalone(text_docs[i], chunk_size=512, chunk_overlap=50)[0] for i in range(len(text_docs))]

table_index = VectorStoreIndex.from_documents(table_docs)
text_index = VectorStoreIndex(text_nodes)

#persist to disk
table_index.storage_context.persist(persist_dir="../db_stores/table_index")
text_index.storage_context.persist(persist_dir='../db_stores/text_index')


Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 2100.30it/s]


In [114]:
#rebuild storage context from disk
table_index = rebuild_index("../db_stores/table_index")
text_index = rebuild_index("../db_stores/text_index")


In [98]:
table_engine = table_index.as_query_engine()
text_engine = text_index.as_query_engine()


In [99]:
# define tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=table_engine,
        metadata=ToolMetadata(
            name="table_tool",
            description=(
                "Useful for retriving specific context from tables"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=text_engine,
        metadata=ToolMetadata(
            name="text_tool",
            description=(
                "Useful for retriving specific context from plain text"
            ),
        ),
    ),
]

In [100]:
# build agent
agent = OpenAIAgent.from_tools(
    query_engine_tools,
    max_function_calls=1,
    verbose=True,
    system_prompt=f"""\
        Respond in Chinese only.
        
        If you inquire general-purpose questions, refer to the text_tool as a priority.
        For questions involving numbers and figures, refer to the table_tool first.
        If you need to analyze problems demanding a higher degree of rigorous reasoning, such as step-by-step instructions, 
        please utilize both the text_tool and table_tool to synthesize your answer.
        
        You must ALWAYS use at least one of the text_tool or table_tool provided when answering a question; do NOT rely on prior knowledge.\
        """,
)

In [133]:
agent.query('气瓶内的气体不应用尽，乙炔气瓶环境温度在 15~25 度时，剩余压强最少多少MPa？')
# 气瓶内的气体不应用尽，乙炔气瓶环境温度在 15~25 度时，剩余压强最少多少MPa？
# 带电体电压等级为500KV，起重机与带电体净空最小安全距离 是几米？
# 测量低压熔断器和水平排列低压母线电流时有什么注意事项

Added user message to memory: 气瓶内的气体不应用尽，乙炔气瓶环境温度在 15~25 度时，剩余压强最少多少MPa？
=== Calling Function ===
Calling function: text_tool with args: {"input":"乙炔气瓶环境温度在 15~25 度时，剩余压强最少多少MPa"}
Got output: 0.2MPa



Response(response='乙炔气瓶在环境温度在15~25度时，剩余压强最少为0.2MPa。', source_nodes=[NodeWithScore(node=TextNode(id_='17a3b174-7301-4ec3-a25b-d4ac292dd0be', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a72ce49e-4150-4d8c-bec2-ee6c6544742f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='1a5a296c2b991a56e6363e167808f9e7d4530b2c9c275363a3c07683cb20b454'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='f041f9f4-c690-4f1e-b6c4-6a65823fc8d2', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='c34f60b6526036b313fb98fda8f7c62ca45daf08a4f9d7dc21554aa8c1c3ca64')}, text='* 点火时应先开乙炔阀、后开氧气阀，嘴孔不得对人；熄火时顺序相反。发生回火或爆鸣时，应先关乙炔阀，再关氧气阀。\n* 使用中的氧气瓶和乙炔气瓶应垂直固定放置，氧气瓶和乙炔气瓶的距离不得小于5m；气瓶的放置地点不得靠近热源，应距明火10m以外。\n* 气瓶不得靠近热源或在烈日下曝晒。乙炔气瓶使用时必须直立放置，禁止卧放使用。\n* 禁止敲击、碰撞乙炔气瓶。气瓶必须装设专用减压器，不同气体的减压器严禁换用或替用。\n* 氧气软管与乙炔软管禁止混用；软管连接处应用专用卡子卡紧或用软金属丝扎紧。氧气、乙炔气软管禁止沾染油脂。软管不得横跨交通要道或将重物压在其上。软管产生鼓包、裂纹、漏气等现象应切除或更换，不应采用贴补或